<a id="1.1"></a>
<h1 style="background-color:orange;font-family:newtimeroman;font-size:300%;text-align:center;border-radius: 15px 50px;">Introduction</h1>


<img src="https://images.unsplash.com/photo-1457520986485-d19e5f3a046d?ixlib=rb-1.2.1&ixid=MXwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHw%3D&auto=format&fit=crop&w=1352&q=80" alt="Girl in a jacket" width="1000" height="600">

<p style="margin: 0cm 0cm 10pt; line-height: 115%; font-size: 15px; font-family: Calibri, sans-serif; text-align: center;"><span style='font-size: 24px; line-height: 115%; color: black; font-family: "Times New Roman", Times, serif;'>Greetings my fellow Kagglers!</span><span style="font-size: 24px;"><span style="font-family: 'Times New Roman', Times, serif;"><br style="text-align:start;">In the following Kernel we will perform a short EDA section, as well as text vectorization ,dimensionality reduction and blending model results.</span></span></p>
<p style="margin: 0cm 0cm 10pt; line-height: 115%; font-size: 15px; font-family: Calibri, sans-serif; text-align: center;"><span style="font-size: 24px;"><span style="font-family: 'Times New Roman', Times, serif;"><span style="line-height: 115%; color: black;"><br>&nbsp;We will construct a Random Forest model to predict the missing values in the &lsquo;price&rsquo; feature (both to provide feature Kagglers with a reasonable solution for the missing value and as a price category prediction model in general).</span></span></span></p>
<p style="margin: 0cm 0cm 10pt; line-height: 115%; font-size: 15px; font-family: Calibri, sans-serif; text-align: center;"><span style="font-size: 24px;"><span style="font-family: 'Times New Roman', Times, serif;"><span style="line-height: 115%; color: black;">&nbsp;</span></span></span></p>
<p style="text-align: center;"><span style='font-family: "Times New Roman", Times, serif; font-size: 24px;'>&nbsp;</span></p>


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import plotly.express as ex
import plotly.graph_objs as go
import plotly.offline as pyo
from plotly.subplots import make_subplots
pyo.init_notebook_mode()
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import r2_score,mean_squared_error,jaccard_score,f1_score,confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier,AdaBoostRegressor
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor,KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from wordcloud import WordCloud,STOPWORDS
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
import re
import string
import plotly.graph_objs as go
def RMSE(Y,Y_HAT):
    return np.sqrt(mean_squared_error(Y_HAT,Y))


plt.rc('figure',figsize=(20,11))

def set_seed(seed=31415):
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
set_seed()

<a id="1.1"></a>
<h1 style="background-color:orange;font-family:newtimeroman;font-size:200%;text-align:center;border-radius: 15px 50px;">Data Loading And Assessment</h1>


In [ ]:
one_star = pd.read_csv('/kaggle/input/michelin-restaurants/one-star-michelin-restaurants.csv')
two_star = pd.read_csv('/kaggle/input/michelin-restaurants/two-stars-michelin-restaurants.csv')
three_star = pd.read_csv('/kaggle/input/michelin-restaurants/three-stars-michelin-restaurants.csv')
one_star['Michelin Stars'] = 1
two_star['Michelin Stars'] = 2
three_star['Michelin Stars'] = 3
r_data = pd.concat([one_star,two_star,three_star],axis=0).drop(columns=['url','zipCode'])
r_data = r_data[~r_data.city.isna()]
r_data

In [ ]:
plt.title('Missing Value Counts In Our Dataset',fontsize=20)
sns.heatmap(pd.DataFrame(r_data.isna().sum()).T,annot=True,cmap='Reds',fmt='d')
plt.show()

<p style="margin: 0cm 0cm 10pt; line-height: 115%; font-size: 15px; font-family: Calibri, sans-serif; text-align: center;"><span style='font-size: 24px; line-height: 115%; color: black; font-family: "Times New Roman", Times, serif;'>So in total we have 176 missing price values out of a total of 693 samples,<br> as explained in the introduction these are our target values for prediction.<br> Also it is important to note that some features were omitted, features such as &lsquo;url&rsquo; and &rsquo;zipCode&rsquo; because of their low relevance as predictor in my opinion.</span></p>

<a id="1.1"></a>
<h1 style="background-color:orange;font-family:newtimeroman;font-size:250%;text-align:center;border-radius: 15px 50px;">Exploratory Data Analysis</h1>


In [ ]:
plt.title('Top 10 Cities With The Most Michelin Restaurants',fontsize=20)
sns.barplot(x=r_data.city.value_counts()[:10],y=r_data.city.value_counts()[:10].index,palette='twilight')
plt.xlabel('Number Of Restaurants',fontsize=18)
plt.show()

<p style="margin: 0cm 0cm 10pt; line-height: 115%; font-size: 15px; font-family: Calibri, sans-serif; text-align: center;"><span style='font-size: 24px; line-height: 115%; color: black; font-family: "Times New Roman", Times, serif;'>In the above plot we observe the top 10 cities in our dataset merged by the amount of Michelin restaurant located in them.</span></p>
<p style="margin: 0cm 0cm 10pt; line-height: 115%; font-size: 15px; font-family: Calibri, sans-serif; text-align: center;"><span style='font-size: 24px; line-height: 115%; color: black; font-family: "Times New Roman", Times, serif;'>And we can see that major size cities such as New York, Hong Kong And San Francisco are leading with the highest count of Michelin restaurant, it is quite expected when taking into mind known facts such as that all those cities belong to first world, developed countries. </span></p>

In [ ]:
plt.title('Top 10 Cuisines With The Most Michelin Restaurants',fontsize=20)
sns.barplot(x=r_data.cuisine.value_counts()[:10],y=r_data.cuisine.value_counts()[:10].index,palette='mako')
plt.xlabel('Number Of Restaurants',fontsize=18)
plt.show()

<p style="margin: 0cm 0cm 10pt; line-height: 115%; font-size: 15px; font-family: Calibri, sans-serif; text-align: center;"><span style='font-size: 24px; line-height: 115%; color: black; font-family: "Times New Roman", Times, serif;'>Looking at the above plot we can observe that the cuisines that are the most popular in Michelin Restaurants are surprisingly different from what one may expect, we see that world famous cuisines such as the Italian, French and British are lower in our list.<br>&nbsp;It may be due to the nature of the data collected but it is interesting none the less.</span></p>
<p style="margin: 0cm 0cm 10pt; line-height: 115%; font-size: 15px; font-family: Calibri, sans-serif; text-align: center;"><span style='font-size: 24px; line-height: 115%; color: black; font-family: "Times New Roman", Times, serif;'>&nbsp;</span></p>

In [ ]:
ex.pie(r_data,names='Michelin Stars',title='Proportion Of Resturants According To Michelin Starts')

<p style="margin: 0cm 0cm 10pt; line-height: 115%; font-size: 15px; font-family: Calibri, sans-serif; text-align: center;"><span style='font-size: 24px; line-height: 115%; color: black; font-family: "Times New Roman", Times, serif;'>As one would expect most of the samples (almost 80%) in our dataset represent 1 start Michelin Restaurants and a very small amount represent 3 star Michelin Restaurants of course due to the nature of receiving the stars which is more difficult from star to star.</span></p>

In [ ]:
ex.pie(r_data,names='region',title='Proportion Of Different Regions In Our Dataset')

In [ ]:
ex.pie(r_data,names='year',title='Proportion Of Different Years in Our Dataset')

In [ ]:
names = ' '.join(re.findall(r'\w+',' '.join(r_data.name.str.lower())))
names = ' '.join(name for name in names.split(' ') if len(name) > 1)
wc = WordCloud(width=800,height=400,stopwords=STOPWORDS,colormap='coolwarm').generate(names)
plt.title('Most Used Words In Restaurants Names',fontsize=20)
plt.imshow(wc)
plt.axis('off')
plt.show()

<p style="margin: 0cm 0cm 10pt; line-height: 115%; font-size: 15px; font-family: Calibri, sans-serif; text-align: center;"><span style='font-family: "Times New Roman", Times, serif; font-size: 24px;'>We can see that there are definitely some words which repeat across the name of different<br>Michelin Restaurants even though most of those restaurants incorporate unique words even if using one of the more popular words that can be seen in the word cloud above.</span></p>

In [ ]:
r_data

In [ ]:
fig = make_subplots(
    rows=2, cols=2,
    column_widths=[0.6, 0.4],
    row_heights=[0.4, 0.6],
    specs=[[{"type": "scattergeo", "rowspan": 2}, {"type": "scatterpolar", "rowspan": 2}],
           [            None                    , None]])

fig.add_trace(
    go.Scatterpolar(r=r_data['Michelin Stars'].value_counts().values,theta=['1 Star','2 Stars','3 Stars'],name='Number Of Michelin Stars',fill='toself'),
    row=1, col=2
)

fig.add_trace(
    go.Scattergeo(lat=r_data["latitude"],
                  lon=r_data["longitude"],
                  mode="markers",
                  text=r_data['Michelin Stars'],
                  hoverinfo="text",
                  showlegend=False,
                  marker=dict(color="crimson", size=4, opacity=0.8)),
    row=1, col=1
)

fig.update_geos(
    projection_type="natural earth",
    landcolor="white",
    oceancolor="MidnightBlue",
    showocean=True,
    lakecolor="LightBlue",
    projection_rotation_lon=-30,
    projection_rotation_lat=15

)

fig.update_xaxes(tickangle=45)

fig.update_layout(title='Distribution Of Samples Around The World And The Amount Of Stars Given')
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scattergeo(lat=r_data["latitude"],
                  lon=r_data["longitude"],
                  mode="markers",
                  name='Michelin Stars By Size',
                  marker_size =r_data['Michelin Stars']*5,
                  text=r_data['Michelin Stars'],
                  hoverinfo="text",
                  showlegend=True,
                  marker=dict(color="crimson", size=4, opacity=0.8)),
    
)

fig.update_geos(
    projection_type="natural earth",
    landcolor="white",
    oceancolor="MidnightBlue",
    showocean=True,
    lakecolor="LightBlue",
    projection_rotation_lon=-30,
    projection_rotation_lat=15

)

fig.update_layout(title='Michelin Star Resturants Around The World')
fig.show()

<a id="1.1"></a>
<h3 style="background-color:orange;font-family:newtimeroman;font-size:200%;text-align:center;border-radius: 15px 50px;">Data Preprocessing And Text Feature Vectorization</h3>


<p style="margin: 0cm 0cm 10pt; line-height: 115%; font-size: 15px; font-family: Calibri, sans-serif; text-align: center;"><span style='font-family: "Times New Roman", Times, serif; font-size: 24px;'>In the following step we will use the &lsquo;name&rsquo; feature from our dataset in order to make a make a rough prediction of the price category.<br>&nbsp;Those prediction will be used as a feature in our final prediction model (Similar idea to what we do while performing model blending)</span></p>
<p style="margin: 0cm 0cm 10pt; line-height: 115%; font-size: 15px; font-family: Calibri, sans-serif; text-align: center;"><span style='font-family: "Times New Roman", Times, serif; font-size: 24px;'>To perform the following maneuver we will first vectorize the name feature use the CountVectorizer,<br> from which we will receive a sparse matrix of words and there counts, we will than apply an SVD to that sparse matrix in order to reduce the dimensionality of the matrix to a dimension with a reasonable amount of explained variance.</span></p>

In [ ]:
N_COMPONENTS = 300

CV = CountVectorizer()
name_c_matrix = CV.fit_transform(r_data.name)
SVD_T = TruncatedSVD(n_components= N_COMPONENTS)
t_name_matirx = SVD_T.fit_transform(name_c_matrix)

desc_ex_var = np.cumsum(SVD_T.explained_variance_ratio_)

tr1 = go.Scatter(x=np.arange(0,len(desc_ex_var)),y=desc_ex_var,name='Cumulative EV')
tr2 = go.Scatter(x=np.arange(0,len(desc_ex_var)),y=SVD_T.explained_variance_ratio_,name='Individual Component Variance')
fig = go.Figure(data=[tr1,tr2],
          layout=dict(title='Ocean Proximity Explained Variance Ratio Using {} Components'.format(N_COMPONENTS),xaxis_title='# Componenets',yaxis_title='Total Variance Explained'))

fig.show()

name_cv_df = pd.DataFrame(t_name_matirx)

<p style="margin: 0cm 0cm 10pt; line-height: 115%; font-size: 15px; font-family: Calibri, sans-serif; text-align: center;"><span style='font-family: "Times New Roman", Times, serif; font-size: 24px;'>Using 300 components (instead of 931) we still are able to explain more than 70% of the variance which is more than enough for our rough prediction.</span></p>
<p style="margin: 0cm 0cm 10pt; line-height: 115%; font-size: 15px; font-family: Calibri, sans-serif; text-align: center;"><span style='font-family: "Times New Roman", Times, serif; font-size: 24px;'>It is important to note that we are producing this rough prediction in order not to waste the text feature we have especially after we saw that there are recurrent words (in other words we don&rsquo;t have uniformly distributed word count across our samples) &nbsp;</span></p>

In [ ]:
r_data.price = r_data.price.apply(lambda x: len(x) if type(x) is str  else np.nan)
city_encoding_dict = dict(r_data.city.value_counts())
region_encoding_dict = dict(r_data.region.value_counts())
cuisine_encoding_dict = dict(r_data.cuisine.value_counts())
r_data.city = r_data.city.apply(lambda x: city_encoding_dict[x])
r_data.region = r_data.region.apply(lambda x: region_encoding_dict[x])
r_data.cuisine = r_data.cuisine.apply(lambda x: cuisine_encoding_dict[x])

<p style="margin: 0cm 0cm 10pt; line-height: 115%; font-size: 15px; font-family: Calibri, sans-serif; text-align: center;"><span style='font-family: "Times New Roman", Times, serif; font-size: 24px;'>Because of the cardinality of the categorical features (cuisine, city, and region) one-hot encoding is not an option, so the method of encoding I selected was Count encoding where each category was replaced with the amount of times it appears in the dataset.</span></p>

<a id="1.1"></a>
<h1 style="background-color:orange;font-family:newtimeroman;font-size:200%;text-align:center;border-radius: 15px 50px;">Random Forest Model Prediction And Evaluation</h1>


<p style="margin: 0cm 0cm 10pt; line-height: 115%; font-size: 15px; font-family: Calibri, sans-serif; text-align: center;"><span style='font-family: "Times New Roman", Times, serif; font-size: 24px;'>For the Random Forest model prediction we will use a rough estimate that we will predict as planned above using the text decomposed matrix together with the following features : &apos;<u>year</u>&apos;, &apos;<u>city</u>&apos;, &apos;<u>region</u>&apos; ,&apos;<u>cuisine</u>&apos; ,&apos;<u>Michelin Stars</u>&apos;, &apos;<u>price</u>&apos;.<br><br></span></p>
<p style="margin: 0cm 0cm 10pt; line-height: 115%; font-size: 15px; font-family: Calibri, sans-serif; text-align: center;"><span style='font-family: "Times New Roman", Times, serif; font-size: 24px;'>The predictions will be evaluated using the f1-score and the Jaccard index.</span></p>

In [ ]:
X_train = r_data[~r_data.price.isna()][['year','city','region','cuisine','Michelin Stars','price']]
Y_train = X_train.pop('price')
X_name_train = name_cv_df.iloc[X_train.index,:]
X_test  = r_data[r_data.price.isna()][['year','city','region','cuisine','Michelin Stars','price']]
Y_test  = X_test.pop('price')
X_name_test = name_cv_df.iloc[X_test.index,:]

In [ ]:
knn_model = KNeighborsClassifier(n_neighbors=10)
knn_model.fit(X_name_train,Y_train)
name_based_preiction = knn_model.predict(X_name_train)
test_name_based_preiction = knn_model.predict(X_name_test)

text_jacc = jaccard_score(Y_train,name_based_preiction,average='macro')
text_f1 = f1_score(Y_train,name_based_preiction,average='macro')
print('F1 Score {:.2f} | Jaccard Score {:.2f}  Of Text Based Prediction'.format(text_f1,text_jacc))

In [ ]:
plt.title('Train Data Prediction Using Text Data Confision Matrix',fontsize=20)
sns.heatmap(confusion_matrix(Y_train,name_based_preiction),annot=True,fmt='d',cmap='PRGn')
plt.show()

<p style="margin: 0cm 0cm 10pt; line-height: 115%; font-size: 15px; font-family: Calibri, sans-serif; text-align: center;"><span style='font-family: "Times New Roman", Times, serif; font-size: 24px;'>Above are the results of the &ldquo;rough&rdquo; prediction we made using the decomposed text matrix we constructed earlier.<br> We see that we have both horrible F1-Scores and Jaccard Scores but it will do as a rough estimate which will hopefully help our random forest model create stronger basis.</span></p>

In [ ]:
X_train['Text_Based_Prediction'] = name_based_preiction
X_test['Text_Based_Prediction'] = test_name_based_preiction

<p style="margin: 0cm 0cm 10pt; line-height: 115%; font-size: 15px; font-family: Calibri, sans-serif; text-align: center;"><span style='font-size: 24px; font-family: "Times New Roman", Times, serif;'>We add both to the train and the test(our evaluation set of originally missing values) the rough estimation done using the above section.</span></p>

In [ ]:
dt_pipe = Pipeline(steps=[
    ('scale',StandardScaler()),
    ('model',RandomForestClassifier(random_state=42))
])
dt_pipe.fit(X_train,Y_train)
dt_pred = dt_pipe.predict(X_train)

text_jacc = jaccard_score(Y_train,dt_pred,average='macro')
text_f1 = f1_score(Y_train,dt_pred,average='macro')
print('F1 Score {:.2f} | Jaccard Score {:.2f}  Of Overall Prediction On Train Data'.format(text_f1,text_jacc))

In [ ]:
plt.title('Train Data Prediction Confision Matrix',fontsize=20)
sns.heatmap(confusion_matrix(Y_train,dt_pred),annot=True,fmt='d',cmap='coolwarm')
plt.show()

<p style="margin: 0cm 0cm 10pt; line-height: 115%; font-size: 15px; font-family: Calibri, sans-serif; text-align: center;"><span style='font-size: 24px; font-family: "Times New Roman", Times, serif;'>So we trained our Random Forest model using the selected above features together with the rough estimate and we see that we get very comforting metrics , F1-Scroe of <u><strong>0.82 </strong></u>and Jaccard Score of <strong><u>0.7</u></strong><br>(also can be evaluated by looking at the confusion matrix).<br> Overall I would estimate that the model does fairly well even though we did not validate it on train and test partitions.</span></p>

In [ ]:
output = pd.DataFrame({'Prediction':dt_pred,'Actual':Y_train})
output.to_csv('view_prediction.csv',index=False)

fig = make_subplots(
    rows=3, cols=2,subplot_titles=('','Actual','Predictions','Residuals'),
    vertical_spacing=0.09,
    specs=[[{"type": "table","rowspan": 3}     ,{"type": "scatter"}] ,
           [None                               ,{"type": "scatter"}]            ,           
           [None                               ,{"type": "scatter"}]                           
          ]
)

fig.add_trace(
    go.Scatter(
        x=np.arange(0,len(output["Actual"])),
        y=output["Actual"],
        mode="markers",
    ),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(
        x=np.arange(0,len(output["Prediction"])),
        y=output["Prediction"],
        mode="markers",
    ),
    row=2, col=2
)

fig.add_trace(
    go.Scatter(
        x=np.arange(0,len(output["Prediction"])),
        y=output["Prediction"]-output["Actual"],
        mode="markers",
    ),
    row=3, col=2
)

fig.add_trace(
    go.Table(
        header=dict(
            values=['Prediction','Actual'],
            font=dict(size=10),
            align="left"
        ),
        cells=dict(
            values=[output[k].tolist() for k in output.columns],
            align = "left")
    ),
    row=1, col=1
)



fig.add_shape(type="line",
    x0=0, y0=(output["Prediction"]-output["Actual"]).mean(), x1=len(output["Prediction"]), y1=(output["Prediction"]-output["Actual"]).mean(),
    line=dict(
        color="Red",
        width=2,
        dash="dashdot",
    ),
        name='Mean',
        xref='x3', 
        yref='y3'
)

fig.update_layout(
    height=800,
    showlegend=False,
    title_text="Prediction Evaluation",
)

fig.show()

In [ ]:
missing_prediction_df  = r_data[r_data.price.isna()]
missing_prediction_df['price'] = dt_pipe.predict(X_test)
missing_prediction_df.to_csv('Missing_Price_Values_Predictions.csv',index=False)
missing_prediction_df.sample(5)
